In [1]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm

import re

In [2]:
# patients = pd.read_csv('../data/pts.csv')

# patients.columns = [x.lower().replace(' ', '_') for x in patients.columns]

# db = sqlite3.connect('../data/ehr.sqlite')

# patients.to_sql('patients', db, if_exists = 'append', index = False) 

299712

In [3]:
# admission = pd.read_csv('../data/hos/admissions.csv')

# admission.columns = [x.lower().replace(' ', '_') for x in admission.columns]

# db = sqlite3.connect('../data/ehr.sqlite')

# admission.to_sql('admission', db, if_exists = 'append', index = False) 

431231

In [4]:
# diagicd = pd.read_csv('../data/diag_icd.csv')

# diagicd.columns = [x.lower().replace(' ', '_') for x in diagicd.columns]

# db = sqlite3.connect('../data/ehr.sqlite')

# diagicd.to_sql('diagicd', db, if_exists = 'append', index = False) 

4756326

In [ ]:
# subject_id	hadm_id patient

In [25]:
db = sqlite3.connect('../data/roughdb.sqlite')

text_subheading = ['Chief Complaint',
     'Major Surgical or Invasive Procedure',
     'History of Present Illness',
     'Past Medical History',
     'Social History',
     'Family History',
     'Physical Exam',
     'Discharge',
     'Pertinent Results',
     'Brief Hospital Course',
     'Medications on Admission',
     'Discharge Medications',
     'Discharge Disposition',
     'Discharge Diagnosis',
     'Discharge Condition',
     'Discharge Instructions',
     'Followup Instructions', '$']

df_e78 = pd.read_csv('../data/icd/e78.csv')
for chunk in tqdm(pd.read_csv('../data/text/discharge.csv', chunksize = 10000)):
    
    dis = chunk.merge(df_e78, how='inner', on=['subject_id', 'hadm_id']) #to get the rows that match our observations
    
    
    
    dic_list = []
    for j in range (len(dis)):

        dic = {'subject_id': (dis.loc[j, 'subject_id']),
               'hadm_id' : (dis.loc[j, 'hadm_id'])}
        
        list_subheading = re.findall('\n(.+):\n', dis.loc[j,'text'])
        list_subheading = list_subheading+['$']


        for i in range (len(list_subheading)-1):
            if list_subheading[i] in text_subheading:
                text_start = re.search(list_subheading[i] + ":\n", dis.loc[j,'text']).end()
                if i < (len(list_subheading)-2):
                    text_end = re.search(list_subheading[i+1] + ":\n", dis.loc[j,'text']).start()
                else:
                    text_end = re.search(list_subheading[i+1], dis.loc[j,'text']).start() 
                text_parsed = dis.loc[j,'text'][text_start:text_end]

                dic[list_subheading[i]]=text_parsed
        dic_list.append(dic)


    df_text = pd.DataFrame(dic_list) [['subject_id', 'hadm_id',
     'Chief Complaint',
     'Major Surgical or Invasive Procedure',
     'History of Present Illness',
     'Past Medical History',
     'Social History',
     'Family History',
     'Physical Exam',
     'Discharge',
     'Pertinent Results',
     'Brief Hospital Course',
     'Medications on Admission',
     'Discharge Medications',
     'Discharge Disposition',
     'Discharge Diagnosis',
     'Discharge Condition',
     'Discharge Instructions',
     'Followup Instructions']]
   


        # filter it to the subjectId and hadm_id
    df_text.to_sql('text', db, if_exists = 'append', index = False)            # Append the chunk to a referral table

0it [00:00, ?it/s]

AttributeError: 'NoneType' object has no attribute 'start'

In [30]:
list_subheading


# re.search(list_subheading[i+1], dis.loc[j,'text']).start()

['Chief Complaint',
 'Major Surgical or Invasive Procedure',
 'History of Present Illness',
 'Past Medical History',
 'Social History',
 'Family History',
 'Physical Exam',
 'DISCHARGE PHYSICAL EXAM',
 'Pertinent Results',
 '___ CHEST (PA AND LAT)',
 '___ P-MIBI',
 'IMAGING METHOD',
 'IMPRESSION',
 'Brief Hospital Course',
 '# Shortness of breath',
 '# Chest Tightness',
 '# HTN',
 '# CHF',
 '# HTN',
 '# Emergency Contacts',
 'Medications on Admission',
 'Discharge Medications',
 'Discharge Disposition',
 'Facility',
 'Discharge Diagnosis',
 'Discharge Condition',
 'Discharge Instructions',
 'Followup Instructions',
 '$']

In [31]:
dis.loc[j, 'text']

' \nName:  ___          Unit No:   ___\n \nAdmission Date:  ___              Discharge Date:   ___\n \nDate of Birth:  ___             Sex:   F\n \nService: MEDICINE\n \nAllergies: \nNo Known Allergies / Adverse Drug Reactions\n \nAttending: ___.\n \nChief Complaint:\nDyspnea and chest tightness\n \nMajor Surgical or Invasive Procedure:\n___ Pharmacologic nuclear stress test\n \nHistory of Present Illness:\n___ year old female with history of HTN, HLD, hx of CVA, CAD s/p \nBMS to circumflex and POBA (___), on Aspirin and Plavix, CHF \n(EF 45% in ___, diabetes, presenting with acute onset \nshortness of breath and substernal chest tightness since ___ \nevening. \n\nPatient notes that ___ evening had a large seafood dinner \nwhich is not usual for her, and then later around 10pm had acute \nonset of SOB feeling like she could not take deep breaths with \nchest tightness (patient notes this is her "angina"). Denies \npleuritic component to CP, described as central and across lower \nrib c

In [103]:
text_subheading = ['Chief Complaint',
     'Major Surgical or Invasive Procedure',
     'History of Present Illness',
     'Past Medical History',
     'Social History',
     'Family History',
     'Physical Exam',
     'Discharge',
     'Pertinent Results',
     'Brief Hospital Course',
     'Medications on Admission',
     'Discharge Medications',
     'Discharge Disposition',
     'Discharge Diagnosis',
     'Discharge Condition',
     'Discharge Instructions',
     'Followup Instructions']


tt1 = re.split('\n(.+):\n', dis.loc[j,'text'])



# for i in range (len(text_subheading)-1):
#     text_parsed =tt1.pop()

# print(i, text_parsed, len('text_subheading'))

In [104]:
tt1 = re.split('\n(.+):\n', dis.loc[j,'text'])
dic = {}
tt1.pop(0)

while len(tt1) > 0:
    
    key = tt1.pop(0)
    value = tt1.pop(0)

    if key in text_subheading:
        dic[key] = value

In [105]:
dic

{'Chief Complaint': 'Dyspnea and chest tightness\n ',
 'Major Surgical or Invasive Procedure': '___ Pharmacologic nuclear stress test\n ',
 'History of Present Illness': '___ year old female with history of HTN, HLD, hx of CVA, CAD s/p \nBMS to circumflex and POBA (___), on Aspirin and Plavix, CHF \n(EF 45% in ___, diabetes, presenting with acute onset \nshortness of breath and substernal chest tightness since ___ \nevening. \n\nPatient notes that ___ evening had a large seafood dinner \nwhich is not usual for her, and then later around 10pm had acute \nonset of SOB feeling like she could not take deep breaths with \nchest tightness (patient notes this is her "angina"). Denies \npleuritic component to CP, described as central and across lower \nrib cage, persistent since onset, no radiation to the \nshoulders/jaw/back, no diaphoreses. Worsens with activity, \nimproves somewhat with rest. Patient does not it feels like \nother episodes when she then required her stent placement. Took \n

In [95]:
tt1

[' \nName:  ___          Unit No:   ___\n \nAdmission Date:  ___              Discharge Date:   ___\n \nDate of Birth:  ___             Sex:   F\n \nService: MEDICINE\n \nAllergies: \nNo Known Allergies / Adverse Drug Reactions\n \nAttending: ___.\n ',
 'Chief Complaint',
 'Dyspnea and chest tightness\n ',
 'Major Surgical or Invasive Procedure',
 '___ Pharmacologic nuclear stress test\n ',
 'History of Present Illness',
 '___ year old female with history of HTN, HLD, hx of CVA, CAD s/p \nBMS to circumflex and POBA (___), on Aspirin and Plavix, CHF \n(EF 45% in ___, diabetes, presenting with acute onset \nshortness of breath and substernal chest tightness since ___ \nevening. \n\nPatient notes that ___ evening had a large seafood dinner \nwhich is not usual for her, and then later around 10pm had acute \nonset of SOB feeling like she could not take deep breaths with \nchest tightness (patient notes this is her "angina"). Denies \npleuritic component to CP, described as central and acro

In [76]:
# tt1.index('Discharge Instructions')
# i = 'Chief Complaint'
for i in range (len(list_subheading)-1):
#     index = tt1.index(i)
    text_parsed =tt1.pop(i+1)
    
    text_parsed

In [72]:
i = 'Family History'
tt1.index(list_subheading[i]+1)
      
#       len(tt1)

TypeError: list indices must be integers or slices, not str

In [114]:
db = sqlite3.connect('../data/roughdb1.sqlite')

text_subheading = ['Chief Complaint',
     'Major Surgical or Invasive Procedure',
     'History of Present Illness',
     'Past Medical History',
     'Social History',
     'Family History',
     'Physical Exam',
     'Discharge',
     'Pertinent Results',
     'Brief Hospital Course',
     'Medications on Admission',
     'Discharge Medications',
     'Discharge Disposition',
     'Discharge Diagnosis',
     'Discharge Condition',
     'Discharge Instructions',
     'Followup Instructions', '$']

df_e78 = pd.read_csv('../data/icd/e78.csv')
for chunk in tqdm(pd.read_csv('../data/text/discharge.csv', chunksize = 10000)):
    
    dis = chunk.merge(df_e78, how='inner', on=['subject_id', 'hadm_id']) #to get the rows that match our observations
    
    
    
    dic_list = []
    for j in range (len(dis)):
        tt1 = re.split('\n(.+):\n', dis.loc[j,'text'])
        dic = {'subject_id': (dis.loc[j, 'subject_id']),
               'hadm_id' : (dis.loc[j, 'hadm_id']),
              'icd' : (dis.loc[j, 'icd']),
              'long_title' : (dis.loc[j, 'long_title'])}
        tt1.pop(0)

        while len(tt1) > 0:
    
            key = tt1.pop(0)
            value = tt1.pop(0)

            if key in text_subheading:
                dic[key] = value

      
 
        dic_list.append(dic)


    df_text = pd.DataFrame(dic_list) [['subject_id', 'hadm_id', 'icd', 'long_title',
     'Chief Complaint',
     'Major Surgical or Invasive Procedure',
     'History of Present Illness',
     'Past Medical History',
     'Social History',
     'Family History',
     'Physical Exam',
     'Discharge',
     'Pertinent Results',
     'Brief Hospital Course',
     'Medications on Admission',
     'Discharge Medications',
     'Discharge Disposition',
     'Discharge Diagnosis',
     'Discharge Condition',
     'Discharge Instructions',
     'Followup Instructions']].rename(columns = {'Chief Complaint': 'chief_complaint',
     'Major Surgical or Invasive Procedure': 'surgical_invasive_procedure',
     'History of Present Illness': 'history_present_illness',
     'Past Medical History': 'past_medcal_history',
     'Social History':'social_history',
     'Family History':'family_history',
     'Physical Exam':'physical_exam',
     'Discharge':'discharge',
     'Pertinent Results': 'pertinent_results',
     'Brief Hospital Course': 'hospital_course',
     'Medications on Admission':'medication_on_admission',
     'Discharge Medications':'medication_on_discharge',
     'Discharge Disposition': 'discharge_disposition',
     'Discharge Diagnosis': 'discharge_diagnosis',
     'Discharge Condition':'discharge_condition',
     'Discharge Instructions':'discharge_instruction',
     'Followup Instructions':'followup_instruction'})
   


        # filter it to the subjectId and hadm_id
    df_text.to_sql('text', db, if_exists = 'append', index = False)            # Append the chunk to a referral table

0it [00:00, ?it/s]

In [48]:
print(tt1[0])
print("-----------")
print(tt1[1])
print("-----------")
print(tt1[2])
print("-----------")
print(tt1[3])
print("-----------")
print(tt1[4])
print("------------")
print(tt1[5])
print("-----------")
print(tt1[6])
print("-----------")
print(tt1[7])
print("-----------")
print(tt1[8])
print("-----------")
print(tt1[9])
print("------------")

 
Name:  ___          Unit No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   F
 
Service: MEDICINE
 
Allergies: 
No Known Allergies / Adverse Drug Reactions
 
Attending: ___.
 
-----------
Chief Complaint
-----------
Dyspnea and chest tightness
 
-----------
Major Surgical or Invasive Procedure
-----------
___ Pharmacologic nuclear stress test
 
------------
History of Present Illness
-----------
___ year old female with history of HTN, HLD, hx of CVA, CAD s/p 
BMS to circumflex and POBA (___), on Aspirin and Plavix, CHF 
(EF 45% in ___, diabetes, presenting with acute onset 
shortness of breath and substernal chest tightness since ___ 
evening. 

Patient notes that ___ evening had a large seafood dinner 
which is not usual for her, and then later around 10pm had acute 
onset of SOB feeling like she could not take deep breaths with 
chest tightness (patient notes this is her "angina"). Denies 
pleuritic component to CP, describ

In [115]:
query = """
SELECT * 
FROM text 
WHERE subject_id = '10000980'
"""
# query = """
# SELECT COUNT (*) 
# FROM text 
# """

with sqlite3.connect('../data/roughdb1.sqlite') as db: 
    text_sqlite = pd.read_sql(query, db)
text_sqlite

,subject_id,hadm_id,icd,long_title,chief_complaint,surgical_invasive_procedure,history_present_illness,past_medcal_history,social_history,family_history,...,discharge,pertinent_results,hospital_course,medication_on_admission,medication_on_discharge,discharge_disposition,discharge_diagnosis,discharge_condition,discharge_instruction,followup_instruction
0,10000980,29654838,E78,Other and unspecified hyperlipidemia,Shortness of breath\n,None\n,"___ yo woman with h/o hypertension, hyperlipid...","1. CAD RISK FACTORS: +Diabetes, +Dyslipidemia,...",___,Denies cardiac family history. Family hx of DM...,...,None,Admission Labs\n___ 01:18PM BLOOD WBC-6.4# RBC...,"___ woman with h/o hypertension, hypelipidemia...",ATENOLOL - 100 mg Tablet - 1.5 Tablet(s) by mo...,1. clopidogrel 75 mg Tablet Sig: One (1) Table...,Home With Service\n,-Hypertension with hypertensive urgency\n-Myoc...,Mental Status: Clear and coherent.\nLevel of C...,"Dear Ms. ___, \n\nYou were admitted for shortn...",___\n
1,10000980,24947999,E78,Other and unspecified hyperlipidemia,Dyspnea and chest tightness\n,___ Pharmacologic nuclear stress test\n,"___ year old female with history of HTN, HLD, ...",- hypertension \n- diabetes \n- hx CVA (cere...,___,Niece had some sort of cancer. Father died in ...,...,None,==== ADMISSION LABS ====\n___ 04:00PM BLOOD WB...,"___ year old female with history of HTN, CVA, ...",The Preadmission Medication list is accurate a...,1. Outpatient Lab Work\nPlease draw Chem-10 an...,Home With Service\n,Primary Diagnosis:\n- Acute on chronic decompe...,Mental Status: Clear and coherent.\nLevel of C...,"Dear ___,\n\n___ were admitted to ___ on ___ w...",___\n
2,10000980,25242409,E78,Other and unspecified hyperlipidemia,DVT\n,EGD\n\n,This is a ___ y/o female with PMHx significant...,- hypertension \n- diabetes \n- hx CVA (cere...,___,Niece had some sort of cancer. Father died in ...,...,None,ADMISSION LABS:\n\n___ 02:45PM BLOOD WBC-5.1 R...,Outpatient Providers: ___ with PMHx significan...,The Preadmission Medication list is accurate a...,1. Allopurinol ___ mg PO EVERY OTHER DAY \n2. ...,Extended Care\n,Primary:\nDeep vein thrombosis (DVT)\n,Mental Status: Clear and coherent.\nLevel of C...,"Dear ___,\n\nIt was a pleasure caring for you ...",___\n
3,10000980,29659838,E78,"Hyperlipidemia, unspecified",dyspnea on exertion\n,None\n\n,"___ with a history of of HTN, CAD s/p DES with...",- hypertension \n- diabetes \n- hx CVA (cere...,___,Father died in his ___ due to lung disease. M...,...,None,ADMISSION LABS\n___ 11:55AM BLOOD WBC-5.4 RBC-...,___ year-old female with history of hypertensi...,The Preadmission Medication list is accurate a...,1. HumuLIN 70/30 (insulin NPH and regular huma...,Home With Service\n,Primary Diagnoses:\nAcute on chronic decompens...,Mental Status: Clear and coherent.\nLevel of C...,"Mrs. ___, \n\n___ were admitted to ___ for tre...",___\n
4,10000980,20897796,E78,"Hyperlipidemia, unspecified",shortness of breath\n,None\n\n,Patient is a ___ with history of coronary arte...,Coronary artery disease\nPeripheral vascular d...,___,Niece had some sort of cancer. Father died in ...,...,None,==============================\n ADMISSION LAB...,None,The Preadmission Medication list is accurate a...,1. Acetaminophen 325-650 mg PO Q6H:PRN Pain -...,Home With Service\n,PRIMARY DIAGNOSIS\n- Acute on chronic diastoli...,Mental Status: Clear and coherent.\nLevel of C...,"Dear ___,\n\nYou were admitted to the hospital...",___\n
